# Structural Estimation HW4

### Takuya Ando

Import packages

In [18]:
# Import packages and load the data
import numpy as np
import pandas as pd
import numpy.random as rnd
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
cmap1 = matplotlib.cm.get_cmap('summer')
# This next command is specifically for Jupyter Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

(a)Load data

In [19]:
macro = pd.read_csv("data/NewMacroSeries.txt", header=None)
macro.columns = ["c_t", "k_t", "w_t", "r_t", "y_t"]
macro.head()

,c_t,k_t,w_t,r_t,y_t
0,1.128323e+07,8.040697e+06,1.120211e+07,1.008852,1.931398e+07
1,1.215464e+07,8.030754e+06,1.206726e+07,1.088112,2.080561e+07
2,1.097303e+07,8.650974e+06,1.089414e+07,0.911904,1.878300e+07
3,9.711635e+06,7.809971e+06,9.641815e+06,0.893986,1.662382e+07
4,9.245673e+06,6.912184e+06,9.179203e+06,0.961637,1.582621e+07


First, we define function that draws N x S values from a normal distribution.

In [62]:
# Define function that draws N x S values from a normal distribution
def norm_draws(unif_vals, mu, sigma):
    '''
    
    '''
    norm_draws = sts.norm.ppf(unif_vals, loc=mu, scale=sigma)
    
    return norm_draws

Then, we define another function which generate simulation values for z, k, w, r, c and y.

In [63]:
def sim_draw(unif_vals, k, alpha, beta, mu, sigma, p, T, S):
    '''
    '''
    draws = norm_draws(unif_vals, mu, sigma)
    
    # z simulation draw
    z_sim = np.zeros((T,S))

    z_sim[0,:] = p * mu + (1-p) * mu + draws[0, :]

    for i in range(1, len(z_sim)):
        z_sim[i,:] = p * z_sim[i-1,:] + (1-p) * mu + draws[i,:]
    
    # k simulation draw
    k_sim = np.zeros((T+1, S))

    k_sim[0] = np.mean(k)

    for i in range(1, T+1):
        k_sim[i,:]=alpha*beta*np.exp(z_sim[i-1,:])*((k_sim[i-1,:])**alpha)
    
    # w, r simulation draw
    w_sim = np.zeros((T, S))

    r_sim = np.zeros((T, S))

    for i in range(T):
        w_sim[i, :] = (1-alpha)*np.exp(z_sim[i,:])*((k_sim[1,:])**alpha)
        r_sim[i, :] = alpha*np.exp(z_sim[i,:])*((k_sim[1,:])**(alpha-1))
    
    # c simulation draw
    c_sim = np.zeros((T, S))

    for i in range(T):
        c_sim[i,:] = w_sim[i, :] + np.multiply(r_sim[i, :], k_sim[i, :]) - k_sim[i+1, :]
    
    # y simulation draw
    y_sim = np.zeros((T, S))
   
    for i in range(T):
        y_sim[i, :]=np.exp(z_sim[i,:])*((k_sim[i,:])**alpha)
    
    
    k_sim = k_sim[0:100, :]
    
    return z_sim, k_sim, w_sim, r_sim, c_sim, y_sim

Then, we degine moment function both for data and model. 

In [64]:
def moments(k, w, r, c, y, data=True):
    '''
    '''
    if data:
        mean_c = np.mean(c)
        mean_k = np.mean(k)
        mean_c_y = np.mean(np.divide(c, y))
        var_y = np.var(y)
        c_t = c[1:100]
        c_t_1 = c[0:99]
        corr_c = np.corrcoef(c_t, c_t_1)[1, 0]
        corr_ck = np.corrcoef(c, k)[1, 0]
    else:
        mean_c = np.mean(c, axis=0)
        mean_k = np.mean(k, axis=0)
        mean_c_y = np.mean(np.divide(c, y), axis=0)
        var_y = np.var(y, axis=0)
        c_t = c[1:100, :]
        c_t_1 = c[0:99, :]
        corr_c = np.zeros(1000)
        for i in range(1000):
            corr_c[i] = np.corrcoef(c_t[:,i], c_t_1[:,i])[1, 0]
        corr_ck = np.zeros(1000)
        for i in range(1000):
            corr_ck[i] = np.corrcoef(c[:,i], k[:,i])[1, 0]
    
    return mean_c, mean_k, mean_c_y, var_y, corr_c, corr_ck

Also, we define error vector and criterion functions.

In [69]:
def err_vec(data_vals, unif_vals, alpha, beta, mu, sigma, p, T, S, simple):
    '''
    --------------------------------------------------------------------
    This function computes the vector of moment errors (in percent
    deviation from the data moment vector) for SMM.
    --------------------------------------------------------------------
    '''
    k, w, r, c, y = data_vals
    
    z_sim, k_sim, w_sim, r_sim, c_sim, y_sim = \
    sim_draw(unif_vals, k, alpha, beta, mu, sigma, p, T, S)
    
    mom_data1, mom_data2, mom_data3, mom_data4,\
    mom_data5 ,mom_data6, = moments(k, w, r, c, y, data=True)
    
    moms_data = np.array([mom_data1, mom_data2, mom_data3, mom_data4, mom_data5, mom_data6])
    
    mom_sim1, mom_sim2, mom_sim3, mom_sim4,\
    mom_sim5 ,mom_sim6, = moments(k_sim, w_sim, r_sim, c_sim, y_sim, data=False)
    
    moms_model = np.array([np.mean(mom_sim1), np.mean(mom_sim2), np.mean(mom_sim3),\
                           np.mean(mom_sim4), np.mean(mom_sim5), np.mean(mom_sim6)])
    
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    
    return err_vec


def criterion(params, *args):
    '''
    --------------------------------------------------------------------
    This function computes the SMM weighted sum of squared moment errors
    criterion function value given parameter values and an estimate of
    the weighting matrix.
    --------------------------------------------------------------------
    '''
    alpha, mu, sigma, p = params
    beta=0.99
    datavals, unif_vals, T, S, W_hat = args
    err = err_vec(datavals, unif_vals, alpha, beta, mu, sigma, p, T, S, simple=False)
    crit_val = err.T @ W_hat @ err
    
    return crit_val

In [68]:
k = macro.k_t
w = macro.w_t
r = macro.r_t
c = macro.c_t
y = macro.y_t

T = 100
S = 1000

alpha_init = 0.5

z = np.log(r/(alpha_init*(k**(alpha_init-1))))

mu_init = np.mean(z)

sigma_init = np.sqrt(np.var(z))

p_init = 0.01

params_init1 = np.array([alpha_init, mu_init, sigma_init, p_init])

datavals = (k, w, r, c, y)
np.random.seed(1)
unif_vals = sts.uniform.rvs(0, 1, size=(T, S)) 

W_hat1 = np.eye(6)
smm_args1 = (datavals, unif_vals, T, S, W_hat1)
results1 = opt.minimize(criterion, params_init1, args=(smm_args1),
                          method='TNC',
                          bounds=((0.01, 0.99), (5, 14), (0.01, 1.1),(-0.99, 0.99)), options={"maxiter":200})
alpha_SMM1, mu_SMM1, sig_SMM1, p_SMM1 = results1.x
print('alpha_SMM1=', alpha_SMM1, ' mu_SMM1=', mu_SMM1, ' sig_SMM1=', sig_SMM1, ' p_SMM1=', p_SMM1)
results1

1.4155234004278327e+29
1.4155234004278327e+29
1.4155273155880006e+29
1.4155236292317062e+29
1.415523592942037e+29
1.4155244124412485e+29
1.4155136295233797e+29
1.4155136295233797e+29
1.4155175446561819e+29
1.4155138583256625e+29
1.4155138220362591e+29
1.4155146415297267e+29
1.415523400383328e+29
1.415523400383328e+29
1.4155273155434957e+29
1.415523629187202e+29
1.4155235928975317e+29
1.4155244123967236e+29
5.293482259404641e+28
5.293482259404641e+28
5.2934967315244605e+28
5.293483110194213e+28
5.293482981816301e+28
5.293486004525562e+28
4.487727804873354e+26
4.487727804873354e+26
4.4877393900433705e+26
4.487728506265121e+26
4.487728428627521e+26
4.487730819689638e+26
4.48769858924704e+26
4.48769858924704e+26
4.4877101743407184e+26
4.4876992906342685e+26
4.4876992129971615e+26
4.487701604043516e+26
4.487727803452792e+26
4.487727803452792e+26
4.487739388622812e+26
4.487728504844622e+26
4.487728427206959e+26
4.487730818269078e+26
1.67300599969228e+26
1.67300599969228e+26
1.673010266858887

2.1987811921386675
2.1987811667968864
2.2900949619870783
2.2900949619870783
2.290094787713118
2.290094944188822
2.2900950135709954
2.2900948984121436
2.1966140550504236
2.1966140550504236
2.1966139505911975
2.1966140455200187
2.1966141060784325
2.1966140234378995
2.185663257688864
2.185663257688864
2.1856632012933965
2.1856632541129164
2.185663303681033
2.185663246743871
2.185663256954483
2.185663256954483
2.1856632005590133
2.1856632533785354
2.1856633029466526
2.1856632460094896
2.1565035662113083
2.1565035662113083
2.1565034804978973
2.1565035591774713
2.1565036197052865
2.156503549949986
2.0242345959471826
2.0242345959471826
2.0242344129435055
2.0242345775656614
2.024234707814806
2.024234563342783
1.7859331795370041
1.7859331795370041
1.785932990675913
1.7859331593034509
1.7859334117436296
1.785933147339575
1.7859331772189135
1.7859331772189135
1.7859329883578223
1.7859331569853605
1.7859334094255426
1.7859331450214853
1.7859331793772828
1.7859331793772828
1.7859329905161918
1.7859

1.336905197041193
1.3369051222179706
1.2011490488104315
1.2011490488104315
1.2011491693346552
1.2011490632214576
1.2011491878207943
1.2011490574679675
1.187923981637269
1.187923981637269
1.1879239966176205
1.1879239841751095
1.1879241069926638
1.1879239666417525
4.941654469369062
4.941654469369062
4.941654415271544
4.941654475263768
4.941653206776138
4.941654784558079
1.9635840958252708
1.9635840958252708
1.9635839137556863
1.963584074629424
1.9635842782517792
1.9635840728069918
1.5018385104639975
1.5018385104639975
1.5018382798853007
1.5018384837802732
1.501838604420888
1.5018384526020778
1.2792621412425254
1.2792621412425254
1.2792619402855285
1.2792621187659423
1.2792622389172634
1.2792620904322676
1.2008253661956916
1.2008253661956916
1.2008252342514232
1.2008253519537755
1.2008254750715806
1.2008253259660593
1.1833692857398468
1.1833692857398468
1.183369215777572
1.1833692786442065
1.1833694022830208
1.1833692555125184
1.1825367466474528
1.1825367466474528
1.1825367161861315
1.182

     fun: 1.1244934682122854
     jac: array([ 1.93412619e-02, -4.15161239e-03,  1.67016490e+01,  3.61759511e-03])
 message: 'Max. number of function evaluations reached'
    nfev: 200
     nit: 38
  status: 3
 success: False
       x: array([ 0.39635652,  5.24105853,  0.01      , -0.04310668])

(b)First, we construct the error matrix function.

In [70]:
def get_Err_mat(datavals, unif_vals, alpha, beta, mu, sigma, p, simple=False):
    '''
    --------------------------------------------------------------------
    This function computes the R x S matrix of errors from each
    simulated moment for each moment error. In this function, we have
    hard coded R = 6.
    --------------------------------------------------------------------
    '''
    R = 6
    S = 1000
    T = 100
    Err_mat = np.zeros((R, S))
    k, w, r, c, y = datavals
    
    z_sim, k_sim, w_sim, r_sim, c_sim, y_sim = \
    sim_draw(unif_vals, k, alpha, beta, mu, sigma, p, T, S)
    
    mom_data1, mom_data2, mom_data3, mom_data4,\
    mom_data5 ,mom_data6, = moments(k, w, r, c, y, data=True)
    
    mom_sim1, mom_sim2, mom_sim3, mom_sim4,\
    mom_sim5 ,mom_sim6, = moments(k_sim, w_sim, r_sim, c_sim, y_sim, data=False)
    
    if simple:
        Err_mat[0, :] = mom_sim1 - mom_data1
        Err_mat[1, :] = mom_sim2 - mom_data2
        Err_mat[2, :] = mom_sim3 - mom_data3
        Err_mat[3, :] = mom_sim4 - mom_data4
        Err_mat[4, :] = mom_sim5 - mom_data5
        Err_mat[5, :] = mom_sim6 - mom_data6
        
        
    else:
        Err_mat[0, :] = mom_sim1 - mom_data1 / mom_data1
        Err_mat[1, :] = mom_sim2 - mom_data2 / mom_data2
        Err_mat[2, :] = mom_sim3 - mom_data3 / mom_data3
        Err_mat[3, :] = mom_sim4 - mom_data4 / mom_data4
        Err_mat[4, :] = mom_sim5 - mom_data5 / mom_data5
        Err_mat[5, :] = mom_sim6 - mom_data6 / mom_data6
    
    return Err_mat

Then, we will construct optimal weighting matrix using the parameters from (a). the optimal weighting matrix is as follows.

In [71]:
Err_mat = get_Err_mat(datavals, unif_vals, alpha_SMM1, 0.99, mu_SMM1, sig_SMM1, p_SMM1, False)
VCV = (1 / S) * (Err_mat @ Err_mat.T)
print(VCV)
# Because VCV4 is poorly conditioned we use the pseudo-inverse to invert it, which
# uses the SVD
W_hat2 = lin.pinv(VCV)
print(W_hat2.shape)

[[ 6.69877348e+13  4.25863257e+13 -3.11789510e+06  2.33595198e+18
  -2.11693330e+06 -1.92690786e+06]
 [ 4.25863257e+13  2.70735778e+13 -1.98215645e+06  1.48492376e+18
  -1.34586273e+06 -1.22494644e+06]
 [-3.11789510e+06 -1.98215645e+06  1.45121815e-01 -1.08702485e+11
   9.85428310e-02  8.96757784e-02]
 [ 2.33595198e+18  1.48492376e+18 -1.08702485e+11  8.18784462e+22
  -7.34798569e+10 -6.72768748e+10]
 [-2.11693330e+06 -1.34586273e+06  9.85428310e-02 -7.34798569e+10
   6.88554970e-02  6.15002182e-02]
 [-1.92690786e+06 -1.22494644e+06  8.96757784e-02 -6.72768748e+10
   6.15002182e-02  5.57876695e-02]]
(6, 6)


Then we perform SMM using optimal weighting matrix. The estimated parameters and minimized criterion function values are as below.

In [72]:
params_init2 = np.array([alpha_init, mu_init, sigma_init, p_init])

smm_args2 = (datavals, unif_vals, T, S, W_hat2)
results2 = opt.minimize(criterion, params_init2, args=(smm_args2),
                          method='TNC',
                          bounds=((0.01, 0.99), (5, 14), (0.01, 1.1),(-0.99, 0.99)))
alpha_SMM2, mu_SMM2, sig_SMM2, p_SMM2 = results2.x
print('alpha_SMM2=', alpha_SMM2, ' mu_SMM2=', mu_SMM2, ' sig_SMM2=', sig_SMM2, ' p_SMM2=', p_SMM2)
print(results2)

alpha_SMM2= 0.4539689750255667  mu_SMM2= 9.19900778225856  sig_SMM2= 0.5269731514680115  p_SMM2= -0.7681318551368129
     fun: 5.333573921412702e-06
     jac: array([ 4.76549583e-05,  6.47534284e-06, -2.65110162e-05,  2.04268368e-05])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 88
     nit: 18
  status: 1
 success: True
       x: array([ 0.45396898,  9.19900778,  0.52697315, -0.76813186])
